# SyftBox SDK Development Testing

This notebook is for testing and validating the SyftBox SDK during development.

In [1]:
%%capture output --no-stderr

# Install the SDK in development mode (run this once)
!pip install nest_asyncio
!pip install -e .

In [2]:
# Import the SDK
import syft_nsai_sdk as sdk
import nest_asyncio
import httpx
import logging
import os

In [3]:
# import client
from syft_nsai_sdk import SyftBoxClient
from syft_nsai_sdk.utils.logger import get_logger
from IPython.display import display, HTML, Markdown

In [4]:
# define async
nest_asyncio.apply()

# initialize logger
logger = get_logger()
logger.success("Logger initialized successfully!")

# Turn off debug logging for your SDK
logging.getLogger('syft_nsai_sdk').setLevel(logging.INFO)

2025-08-28 14:44:14 - syft_nsai_sdk - SUCCESS - Logger initialized successfully!


In [5]:
# initialize client
client = SyftBoxClient()
logger.info(f"Syftbox client created: {client}")

2025-08-28 14:44:16 - syft_nsai_sdk.main - INFO - Accounting configured for callis@openmined.org
2025-08-28 14:44:16 - syft_nsai_sdk.main - INFO - SyftBoxClient initialized for guest@syft.org
2025-08-28 14:44:16 - syft_nsai_sdk - INFO - Syftbox client created: <syft_nsai_sdk.main.SyftBoxClient object at 0x10dfd2d80>
2025-08-28 14:44:16 - syft_nsai_sdk - INFO - SYFTBOX_ACCOUNTING_EMAIL: callis@openmined.org
2025-08-28 14:44:16 - syft_nsai_sdk - INFO - SYFTBOX_ACCOUNTING_PASSWORD: True
2025-08-28 14:44:16 - syft_nsai_sdk.main - INFO - Using syftbox accounting service URL: https://syftaccounting.centralus.cloudapp.azure.com/
2025-08-28 14:44:16 - syft_nsai_sdk.main - INFO - Accounting configured for callis@openmined.org
2025-08-28 14:44:16 - syft_nsai_sdk.main - INFO - Accounting setup successful


In [ ]:
# get env vars
email = os.getenv('SYFTBOX_ACCOUNTING_EMAIL')
password = os.getenv('SYFTBOX_ACCOUNTING_PASSWORD')
logger.info(f"SYFTBOX_ACCOUNTING_EMAIL: {email}")
logger.info(f"SYFTBOX_ACCOUNTING_PASSWORD: {bool(password)}")

# Setup accounting for paid models
await client.setup_accounting(email, password)

In [6]:
# STEP 2: Discover models explicitly 
models = client.discover_models(service_type="chat")
print(f"Found {len(models)} models")

2025-08-28 14:44:17 - syft_nsai_sdk.discovery.scanner - INFO - Found 10 models across 256 datasites
2025-08-28 14:44:17 - syft_nsai_sdk.discovery.scanner - INFO - Cached 10 models from 8 owners
2025-08-28 14:44:17 - syft_nsai_sdk.services.health - WARNING - Unexpected error in health check for keelan-test-3: AUTHENTICATION_ERROR: No accounting service URL configured
2025-08-28 14:44:17 - syft_nsai_sdk.services.health - WARNING - Unexpected error in health check for carl-model: AUTHENTICATION_ERROR: No accounting service URL configured
2025-08-28 14:44:17 - syft_nsai_sdk.services.health - WARNING - Unexpected error in health check for carl-free: AUTHENTICATION_ERROR: No accounting service URL configured
2025-08-28 14:44:17 - syft_nsai_sdk.services.health - WARNING - Unexpected error in health check for claude-sonnet-3.5: AUTHENTICATION_ERROR: No accounting service URL configured
2025-08-28 14:44:17 - syft_nsai_sdk.services.health - WARNING - Unexpected error in health check for deepseek

Found 6 models


In [7]:
print(f"{'Name':<20} {'Owner':<25} {'Tags':<25} {'Services':<25}")
print("-" * 100)
for model in models:
    enabled_services = [s.type.value for s in model.services if s.enabled]
    tags_str = ", ".join(model.tags[:2])
    services_str = ", ".join(enabled_services)
    print(f"{model.name:<20} {model.owner:<25} {tags_str:<25} {services_str:<25}")

Name                 Owner                     Tags                      Services                 
----------------------------------------------------------------------------------------------------
keelan-test-3        kj@kj.dev                 test                      chat, search             
carl-model           callis@openmined.org      Chat, Search              chat, search             
carl-free            callis@openmined.org      chat, search              chat, search             
claude-sonnet-3.5    aggregator@openmined.org  claude, anthropic         chat, search             
deepseek-v3          aggregator@openmined.org  deepseek, opensource      chat                     
test                 irina@openmined.org       test                      chat, search             


In [8]:
print(await client.get_account_info())

{'email': 'callis@openmined.org', 'balance': 120.1, 'currency': 'USD'}


In [9]:
# model = client.find_model("claude-sonnet-3.5", "aggregator@openmined.org")
# model
# response = await client.chat(
#     model_name="carl-model",
#     owner="callis@openmined.org", 
#     prompt="Hello! What is API?",
#     temperature=0.7,
#     max_tokens=200
# )
# Chat with a model
response = await client.chat(
    model_name="claude-sonnet-3.5",
    owner="aggregator@openmined.org",
    prompt="Hello! What is syftbox?",
    temperature=0.7,
    max_tokens=200
)

2025-08-28 14:44:20 - syft_nsai_sdk.main - INFO - Discovered 1 models (health_check=False)
2025-08-28 14:44:20 - syft_nsai_sdk.services.chat - INFO - Payload size: 189
2025-08-28 14:44:20 - syft_nsai_sdk.services.chat - INFO - Prompt length: 23


✓ Response received                                                             



In [10]:
logger.info(response)

2025-08-28 14:44:27 - syft_nsai_sdk - INFO - I don't have specific information about a product or service called "syftbox." It could be a software tool, a hardware device, or a proprietary system, but I don't have definitive details about it in my knowledge base. 

If you're looking for information about syftbox, you might want to:
- Check the official website of the company that makes it
- Look for documentation or user guides related to it
- Provide more context about where you encountered this term

If you have more details about what context you encountered "syftbox" in, I'd be happy to try to provide more helpful information.


In [9]:
# Create conversation (enhanced with owner support)
conversation = client.create_conversation(
    model_name="claude-sonnet-3.5",
    owner="aggregator@openmined.org",
)

2025-08-28 14:42:05 - syft_nsai_sdk.main - INFO - Discovered 1 models (health_check=False)


In [10]:
# Clear history when needed
conversation.clear_history()

In [11]:
# Each message remembers full context
response1 = await conversation.send_message("What is SyftBox?")

2025-08-28 14:42:09 - syft_nsai_sdk.services.chat - INFO - Payload size: 149
2025-08-28 14:42:09 - syft_nsai_sdk.services.chat - INFO - Prompt length: 33


Context length: 33 characters
✓ Response received                                                             



In [12]:
# After each response, check the conversation state
logger.info(f"Messages count: {len(conversation.messages)}")
logger.info(f"Context length: {len(conversation._build_context_prompt('test'))}")

2025-08-28 14:42:17 - syft_nsai_sdk - INFO - Messages count: 2
2025-08-28 14:42:17 - syft_nsai_sdk - INFO - Context length: 717


Context length: 717 characters


In [13]:
logger.info(f"Response:\n{response1}")

2025-08-28 14:42:18 - syft_nsai_sdk - INFO - Response:
SyftBox is not a well-known or established technology or product that I'm aware of in my training data. There could be several possibilities:

1. It might be a relatively new or specialized software tool, framework, or platform
2. It could be a proprietary system used within a specific organization
3. It might be related to Syft, which is an open-source platform for privacy-preserving machine learning
4. It could be a typo or variation of another product name

If you're referring to a specific technology or product called SyftBox, could you provide more context about where you encountered this term? That would help me give you more accurate information.


In [23]:
# second conversation
response2 = await conversation.send_message("How does it work?")  # Remembers first question

2025-08-28 14:43:04 - syft_nsai_sdk.services.chat - INFO - Payload size: 1506
2025-08-28 14:43:04 - syft_nsai_sdk.services.chat - INFO - Prompt length: 1378


Context length: 1378 characters
✓ Response received                                                             



In [24]:
# After each response, check the conversation state
logger.info(f"Messages count: {len(conversation.messages)}")
logger.info(f"Context length: {len(conversation._build_context_prompt('test'))}")

2025-08-28 14:43:11 - syft_nsai_sdk - INFO - Messages count: 4
2025-08-28 14:43:11 - syft_nsai_sdk - INFO - Context length: 1545


Context length: 1545 characters


In [25]:
logger.info(f"Response:\n{response2}")

2025-08-28 14:43:12 - syft_nsai_sdk - INFO - Response:
I apologize, but it appears there's a technical issue with the connection to the Claude API. The system is returning error messages rather than providing responses.

These errors indicate a problem with the JSON formatting of the requests being sent to the Claude API endpoint. Specifically, it's showing:

```
Error loading request: 1 validation error for SyftRequest
Invalid JSON: EOF while parsing a value at line 1 column 0
```

This suggests the request is empty or improperly formatted when it reaches the API.

To get assistance with your questions, you might want to:
1. Try refreshing the page or restarting the application
2. Contact technical support for the platform you're using
3. Try again later, as this might be a temporary service issue

Is there something else I can help you with in the meantime?


In [26]:
# third conversation
response3 = await conversation.send_message("Give an example")    # Remembers both previous

2025-08-28 14:43:14 - syft_nsai_sdk.services.chat - INFO - Payload size: 1696
2025-08-28 14:43:14 - syft_nsai_sdk.services.chat - INFO - Prompt length: 1556


Context length: 1556 characters


2025-08-28 14:43:15 - syft_nsai_sdk.services.chat - WARNING - Unexpected response format, using fallback parsing: {'request_id': 'dbfa3cb8-242e-4c4b-8df0-c920155db138', 'data': {'message': {'body': "Error loading request: 1 validation error for SyftRequest\n  Invalid JSON: EOF while parsing a value at line 1 column 0 [type=json_invalid, input_value='', input_type=str]\n    For further information visit https://errors.pydantic.dev/2.11/v/json_invalid", 'created': '2025-08-28T13:43:14.691143Z', 'expires': '2025-08-29T13:43:14.691146Z', 'headers': {}, 'id': 'dbfa3cb8-242e-4c4b-8df0-c920155db138', 'method': '', 'sender': 'aggregator@openmined.org', 'status_code': 400, 'url': 'syft://aggregator@openmined.org/app_data/claude-sonnet-3.5/rpc/chat'}}}


In [19]:
# After each response, check the conversation state
logger.info(f"Messages count: {len(conversation.messages)}")
logger.info(f"Context length: {len(conversation._build_context_prompt('test'))}")

2025-08-28 14:42:44 - syft_nsai_sdk - INFO - Messages count: 4
2025-08-28 14:42:44 - syft_nsai_sdk - INFO - Context length: 1365


Context length: 1365 characters


In [20]:
logger.info(f"Response:\n{response3}")

2025-08-28 14:42:45 - syft_nsai_sdk - INFO - Response:
{'request_id': 'b7a95a2a-a15b-4e54-8845-f8de8e449d14', 'data': {'message': {'body': "Error loading request: 1 validation error for SyftRequest\n  Invalid JSON: EOF while parsing a value at line 1 column 0 [type=json_invalid, input_value='', input_type=str]\n    For further information visit https://errors.pydantic.dev/2.11/v/json_invalid", 'created': '2025-08-28T13:42:39.455808Z', 'expires': '2025-08-29T13:42:39.455813Z', 'headers': {}, 'id': 'b7a95a2a-a15b-4e54-8845-f8de8e449d14', 'method': '', 'sender': 'aggregator@openmined.org', 'status_code': 400, 'url': 'syft://aggregator@openmined.org/app_data/claude-sonnet-3.5/rpc/chat'}}}


In [21]:
# Get conversation summary
summary = conversation.get_conversation_summary()
logger.info(f"Total messages: {summary['total_messages']}")

2025-08-28 14:42:46 - syft_nsai_sdk - INFO - Total messages: 4
